In [ ]:
!pip install Transformers

# New Section

In [ ]:
!pip install datsets transformers[sentencepiece]
!pip install sentencepiece

ERROR: Could not find a version that satisfies the requirement datsets (from versions: none)
ERROR: No matching distribution found for datsets
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import nltk
import random

from transformers import AutoConfig, AutoModel
model_name = "t5-small"
config = AutoConfig.from_pretrained(
    model_name
  )
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True,
  )
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    config=config
  )

In [ ]:
! pip install datasets

from datasets import load_dataset

dataset = load_dataset('Sarthak279/engl-hind')

master = []
for line in dataset['train']['translation']:
    master.append(line['en'])
    master.append(line['hi_ng'])

def gen_training_data():
    return (master[i : i+500]
    for i in range(0, len(master), 500)
    )
tokenizer_training_data = gen_training_data()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
from transformers import AutoTokenizer
tokenizer_base = AutoTokenizer.from_pretrained("bert-base-cased")
tokenizer = tokenizer_base.train_new_from_iterator(tokenizer_training_data, 32128)

In [ ]:
train_file = "h-e.json"

In [ ]:
data_files = {}
data_files["train"] = train_file
# data_files["valid"] = validation_file
raw_datasets = load_dataset(
    "json",
    data_files=data_files
  )

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
source_prefix = "Translate English to Hinglish : "
source_lang = "en"
target_lang = "hi_ng"
max_source_length = 128
max_target_length = 128 # target and source length task dependent (translation, summary etc.)
padding = "max_length" # padding to max length
num_epochs = 4

In [ ]:
def preprocess(source_data):
  inputs = [sample[source_lang] for sample in source_data["translation"]]
  targets = [sample[target_lang] for sample in source_data["translation"]]
  inputs = [source_prefix + inp for inp in inputs]
  model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

  labels = tokenizer(text_target=targets, max_length=max_target_length, padding=padding, truncation=True)
  # replace tokenizer.pad_token_id in the labels by -100 to ignore padding in the loss.
  labels["input_ids"] = [
      [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    ]
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
train_dataset = raw_datasets["train"]
train_dataset = train_dataset.map(preprocess, batched=True, remove_columns="translation")

Map:   0%|          | 0/63488 [00:00<?, ? examples/s]

In [ ]:
from transformers import default_data_collator
data_collator = default_data_collator


In [ ]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 123.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 98.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 45.0 MB/s eta 0:00:00


In [ ]:
pip install accelerate -U

In [ ]:
from transformers import HfArgumentParser
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer


trainer_args_in = {
    'output_dir': 'my-t5-hinglish-translator',
    'overwrite_output_dir' : True,
    'do_train' : True,
    # 'do_valid' : False,
    'per_device_train_batch_size' : 8,
    'num_train_epochs' : num_epochs,
}

#all arguments can be found detailed in the TrainingArguments dataclass

parser = HfArgumentParser((Seq2SeqTrainingArguments))
training_args = parser.parse_dict(trainer_args_in)

trainer = Seq2SeqTrainer(model=model, args=training_args[0], train_dataset=train_dataset, tokenizer=tokenizer, data_collator=data_collator)

train_result = trainer.train(resume_from_checkpoint=None)
trainer.save_model()

Step,Training Loss
500,2.969200
1000,0.853200
1500,0.333400
2000,0.101500
2500,0.031100
3000,0.015400
3500,0.009200
4000,0.005900
4500,0.004600
5000,0.003700


In [ ]:
from transformers import T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained("my-t5-hinglish-translator")  #path to output dir
input_ids = tokenizer("translate English to Hinglish: Definitely share your feedback in the comment section  .", return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print("Test Output : " + tokenizer.decode(outputs[0], skip_special_tokens=True))

Test Output : ज़रूर comment section में अपना feedback share करें. 


In [ ]:
from transformers import T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained("my-t5-hinglish-translator")  #path to output dir
input_ids = tokenizer("translate English to Hinglish: I was waiting for my bag  .", return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print("Test Output : " + tokenizer.decode(outputs[0], skip_special_tokens=True))

Test Output : मैं अपने bag का इंतज़ार कर रहा था.


In [ ]:
from transformers import T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained("my-t5-hinglish-translator")  #path to output dir
input_ids = tokenizer("translate English to Hinglish: So even if it's a big video, I will clearly mention all the products.  ", return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print("Test Output : " + tokenizer.decode(outputs[0], skip_special_tokens=True))

Test Output : तो चाहे video बड़ा क्यों न हो, मैं साफ़ - साफ़ सभी products का उल्लेख करूंगा.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from transformers import T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained("my-t5-hinglish-translator")  #path to output dir
input_ids = tokenizer("translate English to Hinglish: So even if it's a big video, I will clearly mention all the products.", return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print("Test Output : " + tokenizer.decode(outputs[0], skip_special_tokens=True))

Test Output : तो चाहे video बड़ा क्यों न हो, मैं साफ़ - साफ़ सभी products का उल्लेख करूंगा.


In [ ]:
from transformers import T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained("my-t5-hinglish-translator")  #path to output dir
input_ids = tokenizer("translate English to Hinglish: You are listening to music", return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print("Test Output : " + tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Test Output : तुम music सुन रहे हो


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained("my-t5-hinglish-translator")  #path to output dir
input_ids = tokenizer("translate English to Hinglish: She is brushing her teeth", return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print("Test Output : " + tokenizer.decode(outputs[0], skip_special_tokens=True))

Test Output : वह अपने teeth brush कर रही है


In [ ]:
from transformers import T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained("my-t5-hinglish-translator")  #path to output dir
input_ids = tokenizer("translate English to Hinglish: You are learning Spanish", return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print("Test Output : " + tokenizer.decode(outputs[0], skip_special_tokens=True))

Test Output : तुम Spanish सीख रहे हो


In [ ]:
from transformers import T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained("my-t5-hinglish-translator")  #path to output dir
input_ids = tokenizer("translate English to Hinglish: She is taking a bath", return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print("Test Output : " + tokenizer.decode(outputs[0], skip_special_tokens=True))

Test Output : वह bath ले रही है


In [ ]:
from transformers import T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained("my-t5-hinglish-translator")  #path to output dir
input_ids = tokenizer("translate English to Hinglish: I am practicing yoga", return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print("Test Output : " + tokenizer.decode(outputs[0], skip_special_tokens=True))


Test Output : मैं yoga कर रहा हूँ


In [ ]:
from transformers import T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained("my-t5-hinglish-translator")  #path to output dir
input_ids = tokenizer("translate English to Hinglish: He is fixing the car .", return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print("Test Output : " + tokenizer.decode(outputs[0], skip_special_tokens=True))


Test Output : वह car ठीक कर रहा है


In [ ]:

from transformers import T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained("my-t5-hinglish-translator")  #path to output dir
input_ids = tokenizer("translate English to Hinglish: You are listening to music ", return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print("Test Output : " + tokenizer.decode(outputs[0], skip_special_tokens=True))

Test Output : तुम music सुन रहे हो